# 🎬 LOCOMOT.IO Clip Generator

**Two-phase approach:**
1. Scout many games headless (fast, no video) → find most exciting
2. Record only the best game in portrait HD

Output: 1080x1920 portrait clips for TikTok/Reels/Shorts

In [ ]:
# Setup - installs headless Chrome and ffmpeg
!apt-get update -qq && apt-get install -y -qq chromium-chromedriver ffmpeg > /dev/null
!pip install -q playwright imageio
!playwright install chromium --with-deps > /dev/null 2>&1
print("✅ Setup complete")

In [ ]:
import json
import subprocess
import os
import time
import asyncio
import numpy as np
from PIL import Image
import io
from playwright.async_api import async_playwright
from google.colab import files
from IPython.display import Video, display

# Upload your game HTML
print("📤 Upload index.html from locomot-io:")
uploaded = files.upload()
game_file = list(uploaded.keys())[0]
game_path = os.path.abspath(game_file)
print(f"✅ Loaded: {game_file}")

In [ ]:
# === CONFIG ===
SCOUT_GAMES = 10
SCOUT_DURATION = 45
RECORD_DURATION = 60
NUM_CLIPS = 5
WIDTH, HEIGHT = 1080, 1920

# Hook that FORCES isRecording = true so highlights fire
HIGHLIGHT_HOOK = '''
window._startTime = performance.now();
window._highlights = [];
window._totalScore = 0;

const hook = () => {
    if (typeof HighlightRecorder === 'undefined') {
        setTimeout(hook, 500);
        return;
    }
    
    // FORCE recording on so triggerHighlight doesn't bail early!
    HighlightRecorder.isRecording = true;
    
    const orig = HighlightRecorder.triggerHighlight.bind(HighlightRecorder);
    HighlightRecorder.triggerHighlight = function(type, data) {
        const ts = (performance.now() - window._startTime) / 1000;
        let score = 0;
        if (type === 'kill_streak') score = 100 + (data.streak||0) * 40;
        else if (type === 'became_leader') score = 150;
        else if (type === 'long_train') score = 80;
        else score = 50;
        
        window._highlights.push({type, ts, score});
        window._totalScore += score;
        console.log('HL:' + type + ' score=' + score + ' total=' + window._totalScore);
        return orig(type, data);
    };
    console.log('Hooks ready, isRecording forced ON');
};
hook();
'''

print(f"Scout {SCOUT_GAMES} games, record at {WIDTH}x{HEIGHT}")

In [ ]:
# === PHASE 1: SCOUT GAMES ===

async def scout_game(browser, game_num):
    """Run one game headless, return excitement score"""
    context = await browser.new_context(viewport={'width': 640, 'height': 480})
    page = await context.new_page()
    
    await page.goto(f'file://{game_path}')
    await page.wait_for_load_state('networkidle')
    await asyncio.sleep(0.5)
    await page.evaluate(HIGHLIGHT_HOOK)
    
    # Start game
    for sel in ['#ffaBtn', '#teamBtn', '.start-btn']:
        try:
            await page.click(sel, timeout=1000)
            break
        except:
            continue
    
    await asyncio.sleep(0.5)
    await page.evaluate('window._startTime = performance.now()')
    
    # Run game (no recording - fast!)
    await asyncio.sleep(SCOUT_DURATION)
    
    # Get results
    score = await page.evaluate('window._totalScore')
    highlights = await page.evaluate('window._highlights')
    
    await context.close()
    return {'game': game_num, 'score': score, 'highlights': highlights}

async def scout_all():
    print(f"🔍 PHASE 1: Scouting {SCOUT_GAMES} games...\n")
    
    async with async_playwright() as p:
        browser = await p.chromium.launch(headless=True)
        
        results = []
        for i in range(SCOUT_GAMES):
            result = await scout_game(browser, i + 1)
            results.append(result)
            print(f"  Game {i+1}: score={result['score']} ({len(result['highlights'])} highlights)")
        
        await browser.close()
    
    # Find winner
    winner = max(results, key=lambda x: x['score'])
    print(f"\n🏆 Winner: Game {winner['game']} with score {winner['score']}!")
    return results, winner

scout_results, winner = await scout_all()

In [ ]:
# === PHASE 2: RECORD A GOOD GAME ===
import shutil
os.makedirs('recordings', exist_ok=True)

# Target score = 70% of best scouted game
target_score = int(winner['score'] * 0.7)
print(f"🎯 Target score: {target_score} (70% of winner's {winner['score']})")

async def record_until_good():
    """Keep recording games until we get a good one"""
    
    async with async_playwright() as p:
        browser = await p.chromium.launch(headless=True)
        
        attempt = 0
        while attempt < 5:  # Max 5 attempts
            attempt += 1
            print(f"\n📹 Recording attempt {attempt} at {WIDTH}x{HEIGHT}...")
            
            # Portrait HD with video recording
            context = await browser.new_context(
                viewport={'width': WIDTH, 'height': HEIGHT},
                record_video_dir='recordings/',
                record_video_size={'width': WIDTH, 'height': HEIGHT}
            )
            
            page = await context.new_page()
            await page.goto(f'file://{game_path}')
            await page.wait_for_load_state('networkidle')
            await asyncio.sleep(1)
            await page.evaluate(HIGHLIGHT_HOOK)
            
            # Start game
            for sel in ['#ffaBtn', '#teamBtn', '.start-btn']:
                try:
                    await page.click(sel, timeout=1500)
                    break
                except:
                    continue
            
            await asyncio.sleep(1)
            await page.evaluate('window._startTime = performance.now()')
            
            # Record
            print(f"  Recording {RECORD_DURATION}s...")
            await asyncio.sleep(RECORD_DURATION)
            
            # Check score
            score = await page.evaluate('window._totalScore')
            highlights = await page.evaluate('window._highlights')
            video_path = await page.video.path()
            
            await context.close()
            
            print(f"  Score: {score} ({len(highlights)} highlights)")
            
            if score >= target_score:
                print(f"  ✅ Good enough! (target was {target_score})")
                shutil.move(video_path, 'gameplay.webm')
                await browser.close()
                return highlights
            else:
                print(f"  ❌ Below target, trying again...")
                os.remove(video_path)
        
        # Use last attempt anyway
        print("  Using last attempt")
        await browser.close()
        return highlights

highlights = await record_until_good()
print(f"\n✅ Recorded! gameplay.webm ({os.path.getsize('gameplay.webm')/1024/1024:.1f}MB)")

In [ ]:
# === PHASE 3: EXTRACT CLIPS ===
os.makedirs('clips', exist_ok=True)

if not highlights:
    print("⚠️ No highlights - extracting evenly spaced clips")
    highlights = [{'type': 'auto', 'ts': i*10+5, 'score': 0} for i in range(NUM_CLIPS)]

# Sort by score, select top N spaced apart
selected = []
for h in sorted(highlights, key=lambda x: x.get('score', 0), reverse=True):
    ts = h['ts']
    if all(abs(ts - s['ts']) > 8 for s in selected):
        selected.append(h)
        if len(selected) >= NUM_CLIPS:
            break

print(f"🎬 Extracting {len(selected)} portrait HD clips...\n")

clip_files = []
for i, h in enumerate(selected):
    start = max(0, h['ts'] - 5)
    out = f"clips/clip_{i+1:02d}_{h['type']}_{int(h.get('score',0))}pts.mp4"
    
    # ffmpeg slice - keep portrait resolution
    cmd = ['ffmpeg', '-y', '-ss', str(start), '-i', 'gameplay.webm',
           '-t', '9', '-c:v', 'libx264', '-preset', 'fast', '-crf', '23', out]
    subprocess.run(cmd, capture_output=True)
    
    if os.path.exists(out):
        size = os.path.getsize(out) / 1024 / 1024
        print(f"  ✅ {out} ({size:.1f}MB)")
        clip_files.append(out)

print(f"\n🎉 Created {len(clip_files)} portrait HD clips!")

In [ ]:
# === DOWNLOAD ALL ===
import zipfile

with zipfile.ZipFile('locomotio_clips.zip', 'w') as z:
    for f in clip_files:
        z.write(f)
    # Also include full gameplay
    if os.path.exists('gameplay.mp4'):
        z.write('gameplay.mp4')

print(f"📦 locomotio_clips.zip ({os.path.getsize('locomotio_clips.zip') / 1024 / 1024:.1f} MB)")
files.download('locomotio_clips.zip')

In [ ]:
# Download
from google.colab import files
import zipfile

with zipfile.ZipFile('locomotio_clips.zip', 'w') as z:
    for f in saved:
        z.write(f)
files.download('locomotio_clips.zip')